In [1]:
import pandas as pd
from datetime import datetime, timedelta
version_in = 'V0_1'
version_out = 'V0_4'
path_gen = "/home/pol/Escritorio/INVERBIS/2024/CancerColon/"
path_1 = path_gen + "Data/processed_data/CancerColon_"+str(version_in)+".csv"
colon = pd.read_csv(path_1, sep= ';')

In [2]:
print(colon.shape)
colon.columns

(31400, 32)


Index(['NASI seudonimizado', 'Subg. químico terapéutico ATC disp',
       'Principio activo ATC disp', 'Prezo PVP nomen disp', 'FECHA',
       'FECHA_FIN', 'Actividad', 'Hospital', 'Motivo saída', 'PAC',
       'Lugar asistencia', 'Motivo asistencia', 'Prioridade', 'GNA',
       'Actividade CAPNOR', 'Código Prestación', 'Prestación',
       'Tipo vía rápida', 'Detalle tipo actividade', 'Tipo programación',
       'Cola', 'Tipo intervención', 'Orixe PM/Diagnóstico', 'PM/Diagnóstico',
       'Categoría CIE', 'Subcategoría CIE', 'CENTRO', 'SERVICIO', 'MEDICO',
       'MUESTRA', 'DESCRIPCION', 'DATA EXTRACCIÓN'],
      dtype='object')

In [3]:
def no_colonoscopy(df):
    filtro_sin = ~df['Actividade CAPNOR'].isin(['COLONOSCOPIA', 'COLONOSCOPIA VIA RAPIDA', 'COLONOSCOPIA CON BIOPSIA'])
    filtro_con = df['Actividade CAPNOR'].isin(['COLONOSCOPIA', 'COLONOSCOPIA VIA RAPIDA', 'COLONOSCOPIA CON BIOPSIA'])
    print(df.shape)
    print(df['NASI seudonimizado'].nunique())
    colon_colonoscopia = df[filtro_con]
    print(colon_colonoscopia.shape)
    print(colon_colonoscopia['NASI seudonimizado'].nunique())

    colon_sin_colonoscopia = df[filtro_sin]
    print(colon_sin_colonoscopia.shape)
    print(colon_sin_colonoscopia['NASI seudonimizado'].nunique())

In [4]:
no_colonoscopy(colon)

(31400, 32)
473
(621, 32)
409
(30779, 32)
473


In [5]:
colon['Actividad'].unique()

array(['Ingreso Hospitalizacion', 'Preescripcion consultas y pruebas',
       'Cirugia', 'Alta Hospitalizacion',
       'Lista de Espera quirurgica - Inclusion',
       'Dispensación Farmacia Oncologica',
       'Asistencia a punto de Atención Continuada', 'Defunción',
       'Prueba laboratorio (APA)'], dtype=object)

In [6]:
def filter_dataApa(trace, value):
    trace['FECHA'] = pd.to_datetime(trace['FECHA'], format = '%d-%m-%Y %H:%M:%S')
    # Fecha límite para el filtrado, también como datetime
    limit_date = pd.to_datetime(value, format= '%d-%m-%Y %H:%M:%S')

    limit_date_minus_10s = limit_date - pd.Timedelta(seconds=10)
    trace_filtered = trace[trace['FECHA'] >= limit_date_minus_10s] 
    print(value)
    print(trace_filtered['Actividad'].unique())
    print('\n')
    return trace_filtered


def crear_dead_bolean(trace):
    trace['Defuncion'] = True
    return trace


def modify_dead_date(trace):
    max_date = trace['FECHA'].max()
    new_date = max_date + timedelta(days=30)
    new_date = pd.to_datetime(new_date, format= '%d-%m-%Y %H:%M:%S')
    # Cambiar las fechas asociadas a 'Defunción'
    trace.loc[trace['Actividad'] == 'Defunción', ['FECHA', 'FECHA_FIN']] = new_date
    return trace

In [7]:
#recorrer traza a traza
# y para cada traza: mirar la fecha de extracción de ese paciente
#                  -> filtrar filas de ese trace superiores a esa fecha (DATA_EXTRACT)


colon['FECHA'] = pd.to_datetime(colon['FECHA'], format= '%d-%m-%Y %H:%M:%S')
colon['FECHA_FIN'] = pd.to_datetime(colon['FECHA_FIN'], format= '%d-%m-%Y %H:%M:%S')
grouper = colon.groupby('NASI seudonimizado')

dates, traces = [], []
dates_dict = {}
for name, trace in grouper:
    trace['Defuncion'] = False
    if 'Prueba laboratorio (APA)' in trace['Actividad'].unique().tolist():
        print(name)
        df = trace.query(" Actividad == 'Prueba laboratorio (APA)'")
        value = df['FECHA'].values[0]
        dates.append(value)
        dates_dict[name] = value
        
        #!boleano defunciones
        if 'Defunción' in trace['Actividad'].unique().tolist():
            trace = crear_dead_bolean(trace)
            trace = modify_dead_date(trace)
            
        print('FILTRADO!!!\n')

        traces.append(filter_dataApa(trace, value))

colon_filtered = pd.concat(traces)

0067638e545bc471e1f89de1738c1d5c
FILTRADO!!!

2018-08-07T12:54:56.000000000
['Prueba laboratorio (APA)' 'Preescripcion consultas y pruebas'
 'Ingreso Hospitalizacion' 'Cirugia' 'Alta Hospitalizacion']


010b7970230a0be9031b74e8aff8df0f
FILTRADO!!!

2017-11-08T12:49:35.000000000
['Defunción' 'Prueba laboratorio (APA)'
 'Preescripcion consultas y pruebas' 'Alta Hospitalizacion']


0129d3e82128fa38a023dc2b8d2bf445
FILTRADO!!!

2017-03-24T12:57:14.000000000
['Prueba laboratorio (APA)' 'Preescripcion consultas y pruebas'
 'Ingreso Hospitalizacion' 'Cirugia' 'Alta Hospitalizacion']


017f4d4811a20947be153d6f8f49e1bd
FILTRADO!!!

2017-12-04T12:14:47.000000000
['Prueba laboratorio (APA)' 'Preescripcion consultas y pruebas'
 'Lista de Espera quirurgica - Inclusion' 'Ingreso Hospitalizacion'
 'Cirugia' 'Alta Hospitalizacion' 'Dispensación Farmacia Oncologica']


01ca2f0d170fd6ecf91984415350ce2d
FILTRADO!!!

2018-05-22T12:26:42.000000000
['Prueba laboratorio (APA)' 'Preescripcion consultas y prue

In [8]:
list(dates_dict.keys())
name = 'f82846e5031be242c41b4d461e504fd2'

print(colon['NASI seudonimizado'].nunique())
print(colon_filtered['NASI seudonimizado'].nunique())
print(colon_filtered.columns)
colon_filtered.groupby('Actividad')['NASI seudonimizado'].nunique()

473
473
Index(['NASI seudonimizado', 'Subg. químico terapéutico ATC disp',
       'Principio activo ATC disp', 'Prezo PVP nomen disp', 'FECHA',
       'FECHA_FIN', 'Actividad', 'Hospital', 'Motivo saída', 'PAC',
       'Lugar asistencia', 'Motivo asistencia', 'Prioridade', 'GNA',
       'Actividade CAPNOR', 'Código Prestación', 'Prestación',
       'Tipo vía rápida', 'Detalle tipo actividade', 'Tipo programación',
       'Cola', 'Tipo intervención', 'Orixe PM/Diagnóstico', 'PM/Diagnóstico',
       'Categoría CIE', 'Subcategoría CIE', 'CENTRO', 'SERVICIO', 'MEDICO',
       'MUESTRA', 'DESCRIPCION', 'DATA EXTRACCIÓN', 'Defuncion'],
      dtype='object')


Actividad
Alta Hospitalizacion                         435
Asistencia a punto de Atención Continuada    213
Cirugia                                      356
Defunción                                    131
Dispensación Farmacia Oncologica             213
Ingreso Hospitalizacion                      395
Lista de Espera quirurgica - Inclusion        45
Preescripcion consultas y pruebas            469
Prueba laboratorio (APA)                     473
Name: NASI seudonimizado, dtype: int64

In [9]:
id = 'f82846e5031be242c41b4d461e504fd2'
colon_filtered_prueba = colon_filtered[colon_filtered['NASI seudonimizado'] == id]
colon_filtered_prueba['Actividad'].unique()

array(['Prueba laboratorio (APA)', 'Preescripcion consultas y pruebas'],
      dtype=object)

In [10]:
# def boolAttrib_to_EmptyAttrib(df, atrrib):
#     #recorro cada traza del df
#     grouper = df.groupby('NASI seudonimizado')
#     traces = []
#     for name, trace in grouper:
#         values = trace[attrib].dropna().astype(str).unique().tolist()
#         trace[attrib]  = ', '.join(values)
#         traces.append(trace)
        

#     return pd.concat(traces)
    

# attribs = ['Prioridade', 'GNA', 'Actividade CAPNOR', 'Prestación', 'Tipo vía rápida',	'Detalle tipo actividade','Tipo programación']
# for attrib in attribs:
#     colon_filtered = boolAttrib_to_EmptyAttrib(colon_filtered, attrib)


In [11]:
colon_filtered = colon_filtered[colon_filtered['Actividad'] != 'Preescripcion consultas y pruebas']

In [12]:
# important_drugs = ['LAXANTES OSMÓTICOS','LAXANTES FORMADORES DE VOLUMEN', 'ENEMAS']
# colon_filtered = colon_filtered[colon_filtered['Subg. químico terapéutico ATC disp'].isin(important_drugs)]

In [13]:
# #! Función que conteara el numero de veces que va a buscar medicamentos, y cuanto le cuesta en total estos medicamentos
grouper_2 = colon_filtered.groupby('NASI seudonimizado')

colon_filtered['Nº de compras farmaceuticas'] = 0
colon_filtered['Coste total medicamentos'] = 0
for name, trace in grouper_2: # Prezo PVP nomen disp
    cont = 0 
    cont = trace['Actividad'].value_counts()['Dispensación medicamento ATC']
    print(cont)
    trace['Nº de compras farmaceuticas'] = cont
    trace['Coste total medicamentos'] = trace['Prezo PVP nomen disp'].sum()
    



KeyError: 'Dispensación Farmacia Oncologica'

In [ ]:
colon_filtered[['NASI seudonimizado', 'FECHA', 'FECHA_FIN','Actividad', 'Actividade CAPNOR','Subg. químico terapéutico ATC disp', 'Nº de compras farmaceuticas', 'Coste total medicamentos']]

In [ ]:
iterator = colon_filtered.groupby('NASI seudonimizado')

only_traces = []
for name, trace in iterator:
    activ = trace['Actividad'].unique().tolist()
    if len(activ) == 1:
        only_traces.append(name)

print(only_traces)

In [ ]:
colon_filtered['Defuncion'].value_counts()

In [ ]:
colon_filtered['FECHA'].max()

In [ ]:
colon_filtered

In [ ]:
# Eliminar '\n' y '\r' en valores y cabeceras
#colon_filtered = colon_filtered.replace({r'\r': '', r'\n': ''}, regex=True)

In [ ]:
colon_filtered.to_csv(path_gen+"Data/processed_data/CancerColon_basico_filtered"+str(version_out)+".csv", date_format="%d-%m-%Y %H:%M:%S", index=False,  encoding="UTF-8", sep = ';')

In [ ]:
no_colonoscopy(colon_filtered)